In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
# df_market_data.head()

df_market_data.columns

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_data_scaled = StandardScaler().fit_transform(
       df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']]
)

In [ ]:
# Create a DataFrame with the scaled data
crypto_scaled_df = pd.DataFrame(
    crypto_data_scaled,
    columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']
)

# Copy the crypto names from the original data
crypto_scaled_df['crypto_name']= df_market_data.index

# Set the coinid column as index
crypto_scaled_df = crypto_scaled_df.set_index('crypto_name')

# Display sample data
crypto_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11 (19-1, 05)
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values (19-1, 05)
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(crypto_scaled_df)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

# View a sample of the dataframe
elbow_df.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = elbow_df.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)

# Save plot as a figure
hvplot.save(elbow_plot, "elbow_plot.html")

elbow_plot

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on the above chart, the best value for k would be 4. This is because at k=4, it is the last significant bend in the curve before the line is relatively flat for the rest of the k values.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(crypto_scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_original = model.predict(crypto_scaled_df)

# Print the resulting array of cluster values.
print(f'Array of k clusters:{k_original}')

In [ ]:
# Create a copy of the DataFrame
crypto_predictions_df = crypto_scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
crypto_predictions_df['k4']=k_original

# Display sample data
crypto_predictions_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

crypto_clusters_plot = crypto_predictions_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by='k4',
    hover_cols='crypto_name',
    title='Scatter Plot by Crypto Segment: k=4'
)

hvplot.save(crypto_clusters_plot, "crypto_clusters_plot.html")
crypto_clusters_plot


---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca_data = pca.fit_transform(crypto_scaled_df)

# View the first five rows of the DataFrame. 
crypto_pca_data[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_

total_explained_variance = np.cumsum(explained_variance)

print(f'Explained Variance: {explained_variance}, Total Explained Variance: {total_explained_variance}')

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained varianace is Total Explained Variance: [2.6689967  5.15878002 6.42185214]

In [ ]:
crypto_scaled_df_copy = crypto_scaled_df.copy()
crypto_scaled_df_copy.reset_index(inplace=True)

crypto_scaled_df_copy

In [ ]:
# Create a new DataFrame with the PCA data.
crypto_pca_df = pd.DataFrame(
    crypto_pca_data, columns=['PC1','PC2','PC3']
)

# Copy the crypto names from the original data
crypto_pca_df['crypto_name']= df_market_data.index

# Set the coinid column as index
crypto_pca_df = crypto_pca_df.set_index('crypto_name')

# Display sample data
crypto_pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))


In [ ]:
# Create an empty list to store the inertia values
inertia = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(crypto_pca_df)
    inertia.append(model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {
    'k': k,
    'inertia': inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(pca_elbow_data)


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(
    x='k',
    y='inertia', 
    title='Elbow Curve Using PCA Data', 
    xticks=k)

hvplot.save(elbow_plot_pca, "elbow_plot_pca.html")
elbow_plot_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Based on the line plot above, the best k value would be 4, since it is the last signficant elbow before the line becomes more of a plateau.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it appears to be the same.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)


In [ ]:
# Fit the K-Means model using the PCA data
model.fit(crypto_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_pca_clusters = model.predict(crypto_pca_df)

# Print the resulting array of cluster values.
print(crypto_pca_clusters)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_crypto_pca_predictions = crypto_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
df_crypto_pca_predictions['crypto_pca_clusters'] = crypto_pca_clusters

# Display sample data
df_crypto_pca_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_clusters_plot = df_crypto_pca_predictions.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='crypto_pca_clusters',
    title = "Scatter Plot by Crypto PCA Segment: k=4",
    hover_cols="crypto_name"
)

hvplot.save(pca_clusters_plot, "pca_clusters_plot.html")
pca_clusters_plot


### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_plot * elbow_plot_pca


In [ ]:
# Composite plot to contrast the clusters
crypto_clusters_plot * pca_clusters_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** I think that fewer clusters is more conducive to finding a pattern in the data, it seems to pull the data together more and with fewer outliers.